# Multiprocessing in python

Multiprocessing in Python is a way to run multiple processes simultaneously, allowing Python programs to take advantage of multiple CPU cores for true parallelism.

In [1]:
#importing the  module
import multiprocessing

In [13]:
def test():
    print("this is my multiprocessing programme")

if __name__ =="__main__":
    m=multiprocessing.Process(target=test)
    print("this is my main programme starting")
    m.start() #starting the child process
    m.join() # wait until the child process terminates
    print("this is my main programme ending")




this is my main programme starting
this is my multiprocessing programme
this is my main programme ending


### without join()

In [14]:
def test2():
    print("this is my multiprocessing programme")

if __name__ =="__main__":
    m=multiprocessing.Process(target=test)
    print("without join()")
    print("this is my main programme starting")
    m.start() #starting the child process
    print("this is my main programme ending")

without join()
this is my main programme starting
this is my main programme ending
this is my multiprocessing programme

the main program does not wait for test2 to finish — it just continues executing the next lines immediately.

### multiprocessing.Pool()

In [15]:
def square(n):
    return n**2

if __name__=="__main__":
    with multiprocessing.Pool(processes=5) as pool:
        out=pool.map(square,[3,4,5,6,7,8,9,10])
        print(out)

[9, 16, 25, 36, 49, 64, 81, 100]


Pool will distribute the square functions which will be applied to each element of the array in 5 processes parallelly,
this is faster than normal processing.we can adjust the processes number. in short Pool will create a pool of processes and Process will create only one process at a time

### multiprocessing.Queue()

The multiprocessing.Queue is used in Python to share data safely between multiple processes

In [22]:
def producer(q):
    for i in range(5):
        item = f"Item {i}"
        print(f"producer is putting {item} in queue.")
        q.put(item)
    q.put(None)  # to tell consumer to stop

def consumer(q):
    while True:
        item = q.get()
        if item is None:
            print("Consumer Received indication to stop .")
            break
        print(f"consumer gets {item} from queue.")

if __name__ == "__main__":
    queue = multiprocessing.Queue()
    p = multiprocessing.Process(target=producer, args=(queue,)) # Creating producer process
    c = multiprocessing.Process(target=consumer, args=(queue,)) # creating consumer process
    p.start()
    c.start()
    queue.put("\n new item 5") # we  are directly putting it in queue
    p.join()
    c.join()
    print("Main process completed.")


producer is putting Item 0 in queue.
producer is putting Item 1 in queue.
producer is putting Item 2 in queue.
consumer gets Item 0 from queue.producer is putting Item 3 in queue.
consumer gets 
 new item 5 from queue.

consumer gets Item 1 from queue.producer is putting Item 4 in queue.

consumer gets Item 2 from queue.
consumer gets Item 3 from queue.
consumer gets Item 4 from queue.
Consumer Received indication to stop .
Main process completed.


you can see the out of order outputs as not Time() module is used

In [24]:
import multiprocessing
import time

def producer(q):
    for i in range(5):
        item = f"Item {i}"
        print(f"Producer is putting {item} in queue.", flush=True)
        q.put(item)
        time.sleep(1)  # using sleep to delay
    q.put(None)

def consumer(q):
    while True:
        item = q.get()
        if item is None:
            print("Consumer received indication to stop.", flush=True)
            break
        print(f"Consumer gets {item} from queue.", flush=True)
        time.sleep(1)

if __name__ == "__main__":
    queue = multiprocessing.Queue()
    p = multiprocessing.Process(target=producer, args=(queue,))
    c = multiprocessing.Process(target=consumer, args=(queue,))
    p.start()
    c.start()

    p.join()
    c.join()

    print("Main process completed.", flush=True)


Producer is putting Item 0 in queue.
Consumer gets Item 0 from queue.
Producer is putting Item 1 in queue.
Consumer gets Item 1 from queue.
Producer is putting Item 2 in queue.
Consumer gets Item 2 from queue.
Producer is putting Item 3 in queue.
Consumer gets Item 3 from queue.
Producer is putting Item 4 in queue.
Consumer gets Item 4 from queue.
Consumer received indication to stop.
Main process completed.


### Multiprocessing.Array()

multiprocessing.Array() is a shared memory container.

It allows multiple processes to safely read and write to the same array.

Without it, each process would have its own copy of data, and changes wouldn't be shared.

In [31]:
import multiprocessing

def square(index , value ):
    value[index] = value[index] **2

if __name__ == '__main__':
    arr = multiprocessing.Array('i', [2,3,6,7,8,9],lock=True)
    process = []
    for i in range(6) :
        m = multiprocessing.Process(target=square , args = (i ,arr ))
        process.append(m)
        m.start()
    for m in process:
        m.join()
    print(list(arr))

[4, 9, 36, 49, 64, 81]


### multiprocessing.Pipe()

It’s a way to create a communication channel between two processes

In [36]:
import multiprocessing

def sender(conn, msg):
    for i in msg:
        conn.send(i)
    conn.send(None)  # to indicate no more messages
    conn.close()

def receiver(conn):
    while True:
        msg = conn.recv()
        if msg is None:  # termination
            break
        print(msg)
    conn.close()

if __name__ == '__main__':
    msg = ["my name is samrat","I am a mca student","I love biryani"]

    parent_conn, child_conn = multiprocessing.Pipe()

    m1 = multiprocessing.Process(target=sender, args=(child_conn, msg))
    m2 = multiprocessing.Process(target=receiver, args=(parent_conn,))

    m1.start()
    m2.start()

    m1.join()
    m2.join()

    # Close connections in the main process
    parent_conn.close()
    child_conn.close()


my name is samrat
I am a mca student
I love biryani
